In [1]:
import socket
import struct
import cv2
import numpy as np

def main(host_ip):
    server_address = (host_ip, 65432)
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(server_address)
    sock.listen(1)

    print("Server listening on {}:{}".format(*server_address))

    conn, addr = sock.accept()
    print("Connection from", addr)

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam")
        return

    frame_width = 1920
    frame_height = 1080

    # Set the frame width and height
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error: Could not read frame")
                break

            # Ensure the frame is in RGB format (default for OpenCV)
            resized = cv2.resize(frame, (frame_width, frame_height), 
               interpolation = cv2.INTER_LINEAR)

            # Encode frame as JPEG
            ret, jpeg_frame = cv2.imencode('.jpg', resized)
            if not ret:
                print("Error: Could not encode frame as JPEG")
                break

            # Convert JPEG to bytes
            frame_data = jpeg_frame.tobytes()
            frame_size = len(frame_data)
            conn.sendall(struct.pack('!I', frame_size))
            conn.sendall(frame_data)

            # Create a byte array with 10 INT8 values and 10 Float16 values
            int8_values = np.random.randint(-128, 127, 10, dtype=np.int8)
            float16_values = np.random.uniform(-1, 1, 10).astype(np.float16)

            # Pack the byte array
            byte_array = struct.pack('!10b10e', *(int8_values.tolist() + float16_values.tolist()))

            # Send the byte array
            conn.sendall(byte_array)

    finally:
        cap.release()
        conn.close()
        sock.close()

if __name__ == "__main__":
    main('192.168.0.66')


Server listening on 192.168.0.66:65432
Connection from ('192.168.0.83', 33868)


BrokenPipeError: [Errno 32] Broken pipe